<a href="https://colab.research.google.com/github/ibraztech2/Action-Localization/blob/main/Action_Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Action Localization  Web App on Colab`

## Introduction

###  `What is Action Localization ?`
 This the process of Identifying what actions are perform in a Video  and their respective time of occurence. This occur at two level;
1. Temporal  Recognition ( Intra frame recognition of action)
2. Spatio Recogntion (Inter Frame level recogniton of action)

- `Action Localization` is done in this following order:
   -`Object Detection` (Although other object are detected for action reconition and while the subject (Human) action is what is needed for the loacalization, Other Object detected defines what the subject is doing) )
  - `Temporal Action recognition` (performed by the subject)  at a frame level
  -  `Spatio  Action recognition` at in inter frame level
  -  `Loacalization` of those action detected  at the inter frame level

## How to Run This Notebook

- ### `How`
    To Run this Notebook without having any dependencies issues, It is required to run one after the other has finished running while restarting the Session when prompted to ensure required import are made. This Notebook is primarily designed to run on Google Colab with Streamlit as it frontend where user interact with the web app. However it can be modified as suit to run any other Gpu Providing platform.
- ### `Freemium Colab Gpu Policy`
  - Due to the  Colab Gpu Policy, Gpu resources is only available temporaily .
  - plubic link generated will work as far as Colab session is active
- `NB:`
-   Before Runing the last cell. You're Expected to have your `NGROK_AUTH_TOKEN set` up
    - This is done by adding the NGROK_AUTH_TOKEN to the API key section on Colab with the name `NGROK_AUTH_TOKEN`
-  Re-runing the last cell will invalidate the previous  link. This is due to NGROK freemium policy
- It is advisable to use video with less than 30 sec to conserve memory. However it can be longer as required with premium version of colab

#  ***Backend Setup***

## `1. Drive Mount`

In [1]:
# Check the version of cuda compiler available
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
%%time
#Mount drive to get acces to video input

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
CPU times: user 491 ms, sys: 90.1 ms, total: 581 ms
Wall time: 22 s


## `2. MMAction2 installation And Dependencies Resolution`

### 1. Downgrade Pytorch, Numpy, Transformers, Torchvision, Torchaudio to compatible Version


In [3]:
%%time

# !pip uninstall -y torch torchvision torchaudio mmcv mmcv-full mmengine numpy torchao transformers
!pip uninstall -y torch torchvision torchaudio numpy torchao transformers


# Install PyTorch 2.1.0 with CUDA 11.8
!pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: torchao 0.10.0
Uninstalling torchao-0.10.0:
  Successfully uninstalled torchao-0.10.0
Found existing installation: transformers 4.53.1
Uninstalling transformers-4.53.1:
  Successfully uninstalled transformers-4.53.1
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 590.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

CPU times: user 2.51 s, sys: 1.1 s, total: 3.61 s
Wall time: 2min 30s


### 2. Install MMCV and MMengine

In [4]:
%%time

# Install MMCV 2.1.0 and mmengine
!pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html
!pip install mmengine==0.10.3




Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: mmengine
    Found existing installation: mmengine 0.10.7
    Uninstalling mmengine-0.10.7:
      Successfully uninstalled mmengine-0.10.7
CPU times: user 3.8 s, sys: 775 ms, total: 4.57 s
Wall time: 21.9 s


### 3. Install OpenMIM ( command line tool for MMAction and  MMdetection )

In [5]:
%%time

# Install OpenMIM
!pip install -U openmim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 31.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.4 MB/s eta 0:00:0

CPU times: user 1.54 s, sys: 290 ms, total: 1.83 s
Wall time: 22.6 s


### 4. Build MMaction  and MMDetection from source

In [6]:
# Clone MMAction from Github

!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2
!pip install -e .

# Clone MMDetection from Github
%cd /content/
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -r requirements.txt
!pip install -v -e .



Cloning into 'mmaction2'...
remote: Enumerating objects: 22864, done.
remote: Total 22864 (delta 0), reused 0 (delta 0), pack-reused 22864 (from 1)
Receiving objects: 100% (22864/22864), 69.62 MiB | 24.59 MiB/s, done.
Resolving deltas: 100% (16120/16120), done.
/content/mmaction2
Obtaining file:///content/mmaction2
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 116.1 MB/s eta 0:00:00
  Running setup.py develop for mmaction2
/content
Cloning into 'mmdetection'...
remote: Enumerating objects: 38023, done.
remote: Total 38023 (delta 0), reused 0 (delta 0), pack-reused 38023 (from 1)
Receiving objects: 100% (38023/38023), 63.18 MiB | 27.16 MiB/s, done.
Resolving deltas: 100% (26221/26221), done.
/content/mmdetection
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing met

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Obtaining file:///content/mmdetection
  Running command python setup.py egg_info

  A module that was compiled using NumPy 1.x cannot be run in
  NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
  versions of NumPy, modules must be compiled with NumPy 2.0.
  Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

  If you are a user of the module, the easiest solution will be to
  downgrade to 'numpy<2' or try to upgrade the affected module.
  We expect that some modules will need time to support NumPy 2.

  Traceback (most recent call last):  File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "/content/mmdetection/setup.py", line 11, in <module>
      import torch
    File "/usr/local/lib/python3.11/dist-packages/torch/__init__.py", line 1382, in <module>
      from .functional import *  # noqa: F403
    File "/usr/local/lib/python3.1

### 5. Install Numpy compatible Version

In [7]:
!pip install numpy==1.24.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 108.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.16.0 requires transformers, which is not installed.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, which is not installed.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.24.4 which is incompatible.
yfinance 0.2.65 requires requests>=2.31, but you have requests 2.28.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
blosc2 3.5.1 requires numpy>=1.26, but you have nu

### 6. Download  Temporal Segment Network ( Built on Imagenet ) configuration file and  checkpoint file

In [1]:
# downloading the configuration file to the given location

!mim download mmaction2 --config tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb --dest /content/mmaction2/
!mim download mmdet --config rtmdet_tiny_8xb32-300e_coco --dest /content/mmdetection


processing tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb...
tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb_20220906-2692d16c.pth exists in /content/mmaction2
Successfully dumped tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb.py to /content/mmaction2
processing rtmdet_tiny_8xb32-300e_coco...
rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth exists in /content/mmdetection
Successfully dumped rtmdet_tiny_8xb32-300e_coco.py to /content/mmdetection


### 7. Testing Setup for :

#### 1.  MMAction Setup on a demo video

In [2]:
# The demo.mp4 and label_map_k400.txt are both from Kinetics-400
!python /content/mmaction2/demo/demo.py /content/mmaction2/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb.py \
    /content/mmaction2/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb_20220906-2692d16c.pth \
    /content/mmaction2/demo/demo.mp4 /content/mmaction2/tools/data/kinetics/label_map_k400.txt

Loads checkpoint by local backend from path: /content/mmaction2/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb_20220906-2692d16c.pth
07/12 19:33:05 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
07/12 19:33:05 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
The top-5 labels with corresponding scores are:
arm wrestling:  1.0
rock scissors paper:  9.462836601543262e-14
massaging feet:  1.6511641839499387e-14
stretching leg:  2.9603629982365978e-15
opening bottle:  2.2746603930393246e-15


#### 2. MMDetcttion on a Demo picture

In [3]:
!python /content/mmdetection/demo/image_demo.py /content/mmdetection/demo/demo.jpg /content/mmdetection/rtmdet_tiny_8xb32-300e_coco.py --weights /content/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth --device cpu

Loads checkpoint by local backend from path: /content/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

07/12 19:33:14 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
/usr/local/lib/python3.11/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '
/usr/local/lib/python3.11/dist-packages/torch/functional.py:504: UserWarning: 
torch.meshgrid: in an upcomin

## `3.  Local Inference `

### 1. Download Checkpoints  Files for Action recognition (Slowfast acrn) and Object Detection (Faster RCNN )

In [4]:
!wget https://download.openmmlab.com/mmaction/v1.0/detection/acrn/slowfast-acrn_kinetics400-pretrained-r50_8xb8-8x8x1-cosine-10e_ava21-rgb/slowfast-acrn_kinetics400-pretrained-r50_8xb8-8x8x1-cosine-10e_ava21-rgb_20220906-0dae1a90.pth -P checkpoints/
!wget https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth -P checkpoints/

--2025-07-12 19:33:47--  https://download.openmmlab.com/mmaction/v1.0/detection/acrn/slowfast-acrn_kinetics400-pretrained-r50_8xb8-8x8x1-cosine-10e_ava21-rgb/slowfast-acrn_kinetics400-pretrained-r50_8xb8-8x8x1-cosine-10e_ava21-rgb_20220906-0dae1a90.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.23.180, 47.246.23.181, 47.246.23.184, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.23.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 369451301 (352M) [application/octet-stream]
Saving to: ‘checkpoints/slowfast-acrn_kinetics400-pretrained-r50_8xb8-8x8x1-cosine-10e_ava21-rgb_20220906-0dae1a90.pth’

slowfast-acrn_kinet 100%[===================>] 352.34M  16.7MB/s    in 22s     

2025-07-12 19:34:10 (15.7 MB/s) - ‘checkpoints/slowfast-acrn_kinetics400-pretrained-r50_8xb8-8x8x1-cosine-10e_ava21-rgb_20220906-0dae1a90.pth’ saved [369451301/369451301]

--2025-07-12 19:34:10--  https://download.openmmlab.com/mmdetection/v

### 2. Run Local Inference
- input Video Path must be provided here
- output video path can be provided here

In [18]:
!python /content/mmaction2/demo/demo_spatiotemporal_det.py \
  /content/drive/MyDrive/lift.mp4 \
  result.mp4 \
  --config /content/mmaction2/configs/detection/acrn/slowfast-acrn_kinetics400-pretrained-r50_8xb8-8x8x1-cosine-10e_ava21-rgb.py \
  --checkpoint /content/checkpoints/slowfast-acrn_kinetics400-pretrained-r50_8xb8-8x8x1-cosine-10e_ava21-rgb_20220906-0dae1a90.pth \
  --label-map /content/mmaction2/tools/data/ava/label_map.txt \
  --det-config /content/mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py \
  --det-checkpoint /content/checkpoints/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth \
  --device cuda:0


error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evalu

### 3. Write App file and  Requirements file

#### 1. write Requirements File

In [23]:
%%writefile requirements.txt
streamlit
pyngrok

Overwriting requirements.txt


#### 2. Write frontend App file

In [41]:
%%writefile app.py

# Import required library

from pathlib import Path
import tempfile
import os
import subprocess
from pyngrok import ngrok
import streamlit as st



def predict_action(input_video_path):
    """
      Predict Action takes input video path and return path to processed video.
    """

    output_path = input_video_path

    # Define the command as a list
    command = [
        "python", "/content/mmaction2/demo/demo_spatiotemporal_det.py",
        input_video_path,
        output_path,
        "--config", "/content/mmaction2/configs/detection/acrn/slowfast-acrn_kinetics400-pretrained-r50_8xb8-8x8x1-cosine-10e_ava21-rgb.py",
        "--checkpoint", "/content/checkpoints/slowfast-acrn_kinetics400-pretrained-r50_8xb8-8x8x1-cosine-10e_ava21-rgb_20220906-0dae1a90.pth",
        "--label-map", "/content/mmaction2/tools/data/ava/label_map.txt",
        "--det-config", "/content/mmaction2/demo/demo_configs/faster-rcnn_r50_fpn_2x_coco_infer.py",
        "--det-checkpoint",  "/content/checkpoints/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth",
        "--device", "cuda:0"
    ]

    try:
        subprocess.run(command, check=True)
        return output_path
    except subprocess.CalledProcessError as e:
        print("❌ Error running inference:", e)
        return None


st.set_page_config(page_title="Spatio-Temporal Action Localization", layout="centered")

st.title("🎥 Spatio-Temporal Action Localization")
st.markdown("Upload a video, run inference, and download the results.")

#  Upload video
uploaded_video = st.file_uploader("📤 Upload a video file", type=["mp4", "avi", "mov"])

#  Preview video
if uploaded_video:
    st.video(uploaded_video)

    # Save to a temporary file
    temp_video = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
    temp_video.write(uploaded_video.read())
    temp_video_path = temp_video.name

    #  Run button
    if st.button("▶️ Run Action Localization"):

        with st.spinner("Processing video..."):

            # get output path from predict ation funtion
            processed_video_path = predict_action(temp_video_path)

        # Preview processed Video
        if processed_video_path:
            st.success("✅ Inference complete!")
            if st.checkbox("Show Processed Video"):
              st.video(processed_video_path)



            with open(processed_video_path, "rb") as video_file:
                video_bytes = video_file.read()

            # Create a unique key to detect download
            download_key = "download_result_video"

            # Show download button
            if st.download_button(
                label="⬇️ Download Processed Video",
                data=video_bytes,
                file_name= ,
                mime="video/mp4",
                key=download_key
            ):
                # Delete the file after clicking download
                os.remove(processed_video_path)


# If nothing uploaded
else:
    st.warning("Please upload a video to get started.")

Overwriting app.py


### 4. Install Requirements

In [25]:
!pip install -r requirements.txt

# ***Frontend Setup***

##  4. `App Goes Live on Streamlit`

In [40]:
from pyngrok import ngrok
import os
import streamlit as st
import threading
from google.colab import userdata

def close_ngrok_tunnels():
    try:
        for tunnel in ngrok.get_tunnels():
            ngrok.disconnect(tunnel.public_url)
        st.info("Closed existing ngrok tunnels.")
    except Exception as e:
        st.warning(f"Could not close ngrok tunnels: {e}")

close_ngrok_tunnels()  # Close existing tunnels

# Get the authtoken from Colab secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN') # Input  your Ngrok token   get yours @' https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n'

# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


# Function to run Streamlit
def run_streamlit():
    os.system("streamlit run app.py")

# Open a tunnel on the default Streamlit port 8501
public_url = ngrok.connect(8501)
print("Streamlit app is live at:", public_url)

# Run Streamlit in a separate thread
threading.Thread(target=run_streamlit).start()


2025-07-12 20:48:12.723 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 20:48:12.724 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 20:48:12.725 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Streamlit app is live at: NgrokTunnel: "https://db154f50ebc8.ngrok-free.app" -> "http://localhost:8501"
